In [17]:
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv('/content/qa_pairs.csv')

# Inspect data
print(df.head())

# Check for missing values
print(df.isnull().sum())

# Drop rows with missing values if any
df = df.dropna()

# Convert to JSON-like format for instruction tuning
df['data'] = df.apply(lambda row: {'instruction': 'Answer the following question based on your knowledge.',
                                   'question': row['Question'],
                                   'answer': row['Answer']}, axis=1)

# Save to a new JSON file
df['data'].to_json('formatted_dataset.json', orient='records')

# Split data into train, validate, and test sets
train, validate, test = np.split(df.sample(frac=1, random_state=42),
                                 [int(.8*len(df)), int(.9*len(df))])

                                            Question  \
0  What approach did the political parties in Pak...   
1  What is the US government's stance towards Pak...   
2  What is the root cause of the Muslims' resentm...   
3  What was the purpose of the Charter of Democracy?   
4  How did Nawaz Sharif react to a few members of...   

                                              Answer  
0  They aimed to bring peace through dialogue rat...  
1  The US government has decried the peace accord...  
2  The root causes are past and present discrimin...  
3  Nawaz Sharif and late Benazir Bhutto were goin...  
4  Nawaz Sharif thought for a long time and then ...  
Question    0
Answer      0
dtype: int64


In [18]:
train

,Question,Answer,data
291,What is the Deoband seminary's view on terrori...,The prestigious seminary at Deoband has ruled ...,{'instruction': 'Answer the following question...
3056,What were the two instances where the Pakistan...,The Pakistani army was tested twice within a f...,{'instruction': 'Answer the following question...
2093,What are some reasons for corruption in the po...,Low pay and insufficient compensation packages...,{'instruction': 'Answer the following question...
432,What are the potential consequences of an Isra...,The fallout could be radioactive chaos and the...,{'instruction': 'Answer the following question...
479,What is the author's concern about the loss of...,The author is concerned that the loss of the s...,{'instruction': 'Answer the following question...
...,...,...,...
2220,What was the trauma of the Partition?,The refugees on both sides received a poor rec...,{'instruction': 'Answer the following question...
2979,What did the Prime Minister recently commit to...,The PM recently committed to fighting corrupti...,{'instruction': 'Answer the following question...
2105,What is the significance of the Sachar Report ...,The Sachar Report is a high powered report ini...,{'instruction': 'Answer the following question...
133,What is the proposed solution to dealing with ...,Catch the speculators and deal with them as th...,{'instruction': 'Answer the following question...


In [19]:
# Function to save checkpoints
def save_checkpoint(model, optimizer, step, path="checkpoints"):
    if not os.path.exists(path):
        os.makedirs(path)
    checkpoint_path = os.path.join(path, f"checkpoint_{step}.pt")
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, checkpoint_path)
    print(f"Checkpoint saved to {checkpoint_path}")

In [20]:
def encode_data(tokenizer, questions, answers, max_length=128):
    # Check if the tokenizer has a padding token, set if not
    if tokenizer.pad_token is None:
        print("No padding token found. Using `eos_token` as `pad_token`.")
        tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

    return tokenizer(questions, answers, truncation=True, padding='max_length', max_length=max_length)

#**GPT2**

In [21]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from torch.utils.data import DataLoader, Dataset
import torch
import torch.optim as optim

In [22]:
import torch
torch.cuda.is_available()


False

In [25]:
# Initialize tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.to('cpu')  # Ensure model is on GPU if available

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [26]:
# Check and set padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [27]:
class QADataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

In [28]:
# Function to tokenize data
def encode_data(tokenizer, questions, answers):
    return tokenizer(questions, answers, truncation=True, padding='max_length', max_length=128)

In [29]:
# Prepare datasets
train_encodings = encode_data(tokenizer, train['Question'].tolist(), train['Answer'].tolist())
train_dataset = QADataset(train_encodings)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [30]:
# Optimizer
optimizer = optim.AdamW(model.parameters(), lr=5e-5)

In [ ]:
# Training loop
model.train()
step = 0
for epoch in range(3):  # number of epochs
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to('cpu')
        attention_mask = batch['attention_mask'].to('cpu')
        labels = input_ids
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        # print(f"Epoch {epoch}, Loss {loss.item()}")

        if (step + 1) % 1131 == 0:
            save_checkpoint(model, optimizer, step+1)

        step += 1
        print(f"Step {step}, Loss {loss.item()}")


Step 1, Loss 7.071007251739502
Step 2, Loss 4.424864768981934
Step 3, Loss 2.5355098247528076
Step 4, Loss 1.8143633604049683
Step 5, Loss 1.3688706159591675
Step 6, Loss 1.1164625883102417
Step 7, Loss 2.6985533237457275
Step 8, Loss 1.406386137008667
Step 9, Loss 2.0608749389648438
Step 10, Loss 1.488075852394104
Step 11, Loss 1.4531152248382568
Step 12, Loss 1.46669340133667
Step 13, Loss 1.1481595039367676
Step 14, Loss 1.0777345895767212
Step 15, Loss 1.6924307346343994
Step 16, Loss 2.0468809604644775
Step 17, Loss 1.3495584726333618
Step 19, Loss 1.3800315856933594
Step 20, Loss 1.3761084079742432
Step 21, Loss 1.4692143201828003
Step 22, Loss 1.5447944402694702
Step 23, Loss 1.5103892087936401
Step 24, Loss 1.6343141794204712
Step 25, Loss 1.4366356134414673
Step 26, Loss 1.0745160579681396
Step 27, Loss 1.1464712619781494
Step 28, Loss 1.3857817649841309
Step 29, Loss 1.8218520879745483
Step 30, Loss 1.3089367151260376
Step 31, Loss 1.0369764566421509
Step 32, Loss 1.455783486

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Prepare validation and test datasets
validate_encodings = encode_data(tokenizer, validate['Question'].tolist(), validate['Answer'].tolist())
validate_dataset = QADataset(validate_encodings)
validate_loader = DataLoader(validate_dataset, batch_size=4, shuffle=False)  # No need to shuffle

test_encodings = encode_data(tokenizer, test['Question'].tolist(), test['Answer'].tolist())
test_dataset = QADataset(test_encodings)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)  # No need to shuffle

# Function to evaluate model
def evaluate_model(model, data_loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to('cpu')
            attention_mask = batch['attention_mask'].to('cpu')
            labels = input_ids
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=-1).tolist()
            all_preds.extend(preds)
            all_labels.extend(labels.tolist())

    return all_preds, all_labels

# Evaluate on validation set
val_preds, val_labels = evaluate_model(model, validate_loader)
val_precision = precision_score(val_labels, val_preds, average='macro')
val_recall = recall_score(val_labels, val_preds, average='macro')
val_f1 = f1_score(val_labels, val_preds, average='macro')

print("Validation Precision:", val_precision)
print("Validation Recall:", val_recall)
print("Validation F1-score:", val_f1)

# Evaluate on test set
test_preds, test_labels = evaluate_model(model, test_loader)
test_precision = precision_score(test_labels, test_preds, average='macro')
test_recall = recall_score(test_labels, test_preds, average='macro')
test_f1 = f1_score(test_labels, test_preds, average='macro')

print("Test Precision:", test_precision)
print("Test Recall:", test_recall)
print("Test F1-score:", test_f1)
